In [ ]:
import numpy as np
from scipy.optimize import root
from scipy.integrate import odeint
from scipy.linalg import solve_continuous_are
import matplotlib.pyplot as plt

# Problem 1

In [ ]:
def linearized_init(M, m, l, q1, q2, q3, q4, r):
    '''
    Parameters:
    ----------
    M, m: floats
          masses of the rickshaw and the present
    l   : float
          length of the rod
    q1, q2, q3, q4, r : floats
        relative weights of the position and velocity of the rickshaw, the
        angular displacement theta and the change in theta, and the control


    Return
    -------
    A : ndarray of shape (4,4)
    B : ndarray of shape (4,1)
    Q : ndarray of shape (4,4)
    R : ndarray of shape (1,1)
    '''
    G = 9.8
    A = np.array([[0,1.,0,0],[0,0,m*G/M,0],[0,0,0,1.],[0,0,G/(M*l)*(M+m),0]])
    B = np.array([0,1./M,0,1./(M*l)])[:,np.newaxis]
    Q = np.array([[q1,0,0,0],[0,q2,0,0],[0,0,q3,0],[0,0,0,q4]])
    R = np.array([[r]])
    
    return A, B, Q, R

# Problem 2

In [ ]:
def find_P(A,B,Q,Rinv):
    '''
    Parameters:
    ----------
    A, B, Q : ndarrays of shape (4,4)
    R : ndarray of shape (1,1)
    
    Returns
    -------
    P : the matrix solution of the Riccati equation
    '''
    #Rinv = np.linalg.inv(R)
    P = np.ones(16)*.1
    P = np.random.randn(16)
    def f(x):
        x = x.reshape((4,4))
        eqn = x.dot(A) + np.dot(A.T, x) + Q - x.dot(B.dot(Rinv.dot(B.T.dot(x))))
        return eqn.reshape(16)
    sol = root(f, P)['x'].reshape((4,4))
    
    return sol

In [ ]:
M, m = 23., 5.
l = 4.
q1, q2, q3, q4 = 1., 1., 1., 1.
r = 5.

A, B, Q, R = linearized_init(M, m, l, q1, q2, q3, q4, r)
Rinv = np.linalg.inv(R)

P = find_P(A, B, Q, Rinv)

In [ ]:
result = A - B.dot(Rinv.dot(B.T.dot(P)))
print np.linalg.eig(result)[0]
print
print 'There is a positive eigenvalue, so the solution will not go to 0 over time.'

# Problem 3

In [ ]:
def rickshaw(tv, X0, A, B, Q, R_inv, P):
    '''
    Parameters:
    ----------
    tv : ndarray of time values, with shape (n+1,)
    X0 : 
    A, B, Q : ndarrays of shape (4,4)
    R_inv : ndarray of shape (1,1), inverse of R
    P : ndarray of shape (4,4)
    
    Returns
    -------
    Z : ndarray of shape (n+1,4), the state vector at each time
    U : ndarray of shape (n,), the control values
    '''
    
    K = A - B.dot(R_inv.dot(B.T.dot(P)))
    def f(y, t):
        return K.dot(y)
    
    Z = odeint(f, X0, tv)
    U = -R_inv.dot(B.T).dot(P).dot(Z.T).flatten()
    
    return Z, U

# Problem 4

In [ ]:
M, m = 23., 5.
l = 4.
q1, q2, q3, q4 = 1., 1., 1., 1.
r = 10.
tf1 = 7
tf2 = 60
X0 = np.array([-1., -1., .1, -.2])
jumps = 200
tv1 = np.linspace(0, tf1, jumps)
tv2 = np.linspace(0, tf2, jumps)

A, B, Q, R = linearized_init(M, m, l, q1, q2, q3, q4, r)
Rinv = np.linalg.inv(R)
P1 = find_P(A, B, Q, Rinv)
P2 = solve_continuous_are(A, B, Q, R)

Z1, U1 = rickshaw(tv1, X0, A, B, Q, Rinv, P1)
Z2, U2 = rickshaw(tv2, X0, A, B, Q, Rinv, P2)

In [ ]:
plt.plot(tv1, Z1[:,0], label='$x$')
plt.plot(tv1, Z1[:,1], label='$\dot x$')
plt.plot(tv1, Z1[:,2], label='$\\theta$')
plt.plot(tv1, Z1[:,3], label='$\dot \\theta$')
plt.plot(tv1, U1, label='$\~ u$')

plt.xlim(0, 7)
plt.ylim(-60, 20)

plt.legend(loc=0)
plt.show()

plt.plot(tv2, Z2[:,0], label='$x$')
plt.plot(tv2, Z2[:,1], label='$\dot x$')
plt.plot(tv2, Z2[:,2], label='$\\theta$')
plt.plot(tv2, Z2[:,3], label='$\dot \\theta$')
plt.plot(tv2, U2, label='$\~ u$')

plt.xlim(0, 60)
plt.ylim(-6, 6)

plt.legend(loc=0)
plt.show()

# Problem 5

In [ ]:
M, m = 23., 5.
l = 4.
q1, q2, q3, q4 = 1., 1., 1., 1.
r = 10.
tf = 60
jumps = 200
for i in np.linspace(0,.2,5):
    for j in np.linspace(-.4,0,5):
        X0 = np.array([-1, -1, i, j])

        tv = np.linspace(0, tf, jumps)

        A, B, Q, R = linearized_init(M, m, l, q1, q2, q3, q4, r)
        Rinv = np.linalg.inv(R)
        P = solve_continuous_are(A, B, Q, R)

        Z, U = rickshaw(tv, X0, A, B, Q, Rinv, P)
        
        plt.plot(tv, Z[:,0], label='$x$')
        plt.plot(tv, Z[:,1], label='$\dot x$')
        plt.plot(tv, Z[:,2], label='$\\theta$')
        plt.plot(tv, Z[:,3], label='$\dot \\theta$')
        plt.plot(tv, U, label='$\~ u$')
        
        plt.xlim(0, 60)
        plt.ylim(-6, 6)
        
        plt.title('$\\theta={}, \dot\\theta={}$'.format(i,j))
        plt.legend(loc=0)
        plt.show()